In [ ]:
import imageio
import imgaug as ia
from imgaug.augmentables.kps import Keypoint
from imgaug.augmentables.kps import Keypoint, KeypointsOnImage
import imgaug.augmenters as iaa
import numpy as np

ia.seed(3)
%matplotlib inline

In [ ]:
import os
import json
import os.path as osp
from os.path import isdir, join
import re

In [ ]:
dataset_dir = "./hockey_dataset"
SERIES = ["Tripping", "Slashing", "No_penalty"]

In [ ]:
def read_image(image_dir, frame_no):

    frame_name = "frame_%04d.png" % int(frame_no)
    image = imageio.imread(osp.join(image_dir, frame_name))
    image = ia.imresize_single_image(image, (360, 640))
    #         ia.imshow(image)

    return image

In [ ]:
def atoi(text):
    return int(text) if text.isdigit() else text


def natural_keys(text):
    return [atoi(c) for c in re.split(r"(\d+)", text)]


def get_players(pose):

    keys = set().union(*(d.keys() for d in pose))
    keys.remove("frameNum")
    all_players = list(keys)
    all_players.sort(key=natural_keys)

    return all_players

In [ ]:
def get_keypoints(frame_no, players, pose, kps, kps_meta):
    body = pose[frame_no]
    for key in players:
        player_body = body[key]
        for i in range(0, 48, 3):
            x = player_body[i]
            y = player_body[i + 1]
            kps.append(Keypoint(x, y))
            meta = [frame_no, key, i]
            kps_meta.append(meta)

    return kps, kps_meta

In [ ]:
# def flip():

#     seq = iaa.Sequential([
#     iaa.flip.Fliplr(p=1)])
#     seq_det = seq.to_deterministic()

#     return seq_det

In [ ]:
import random


def pers_scale(affine_scale, pers_scale):

    choice = random.randint(0, 1)
    #     print(choice)

    if choice == 0:
        # perspective

        print("perspective" + str(pers_scale))
        seq = iaa.Sequential([iaa.PerspectiveTransform(scale=(pers_scale, pers_scale))])
        seq_det = seq.to_deterministic()

    elif choice == 1:
        # scale
        print("scale" + str(affine_scale))
        seq = iaa.Sequential([iaa.Affine(scale=(affine_scale, affine_scale))])
        seq_det = seq.to_deterministic()

    return seq_det

In [ ]:
def test_draw(frame, frame_aug, kpsoi, kpsoi_aug):
    ia.imshow(
        np.hstack(
            [
                kpsoi.draw_on_image(frame, size=7),
                kpsoi_aug.draw_on_image(frame_aug, size=7),
            ]
        )
    )

In [ ]:
def Augment_2(image_dir, pose):
    keypoints_augment = []
    keypoints_meta = []
    frames_augment = np.zeros((64, 360, 640, 3))

    per_scale = [0.05, 0.06, 0.07, 0.08, 0.09, 0.1]
    affine_sclae = [1.2, 1.3, 1.4, 1.5]

    per_rand = random.randint(0, 5)
    affine_rand = random.randint(0, 3)

    affine_augmenter = pers_scale(
        affine_scale=affine_sclae[affine_rand], pers_scale=per_scale[per_rand]
    )

    for frame_no in range(64):

        kps = []
        kps_meta = []
        frame = read_image(image_dir, frame_no)
        players = get_players(pose)
        kps, kps_meta = get_keypoints(frame_no, players, pose, kps, kps_meta)
        kpsoi = KeypointsOnImage(kps, shape=frame.shape)
        frame_aug, kpsoi_aug = affine_augmenter(image=frame, keypoints=kpsoi)
        #         test_draw(frame, frame_aug, kpsoi, kpsoi_aug )
        frames_augment[frame_no, :, :, :] = frame_aug
        kpsoi_aug = [kpsoi_aug]
        keypoints_augment.append(kpsoi_aug)
        keypoints_meta.append(kps_meta)

    return keypoints_augment, frames_augment, keypoints_meta

In [ ]:
# def Augment(image_dir, pose):
#     keypoints_augment = []
#     keypoints_meta = []
#     frames_augment = np.zeros((64,360,640,3))
#     flip_augmenter = flip()

#     for frame_no in range(64):

#         kps = []
#         kps_meta = []
#         frame = read_image(image_dir, frame_no)
#         players = get_players(pose)
#         kps, kps_meta = get_keypoints(frame_no, players, pose, kps, kps_meta)
#         kpsoi = KeypointsOnImage(kps, shape=frame.shape)
#         frame_aug, kpsoi_aug = flip_augmenter(image=frame, keypoints=kpsoi)
# #         test_draw(frame, frame_aug, kpsoi, kpsoi_aug )
#         frames_augment[frame_no,:,:,:] = frame_aug
#         kpsoi_aug = [kpsoi_aug]
#         keypoints_augment.append(kpsoi_aug)
#         keypoints_meta.append(kps_meta)

#     return keypoints_augment, frames_augment, keypoints_meta

In [ ]:
def write_image(dest_frame_pose_path, frames_augment):

    for frame_no in range(frames_augment.shape[0]):

        image = frames_augment[frame_no, :, :, :]
        image_uint8 = image.astype(np.uint8)
        frame_name = "frame_%04d.png" % int(frame_no)
        imageio.imsave(osp.join(dest_frame_pose_path, frame_name), image_uint8)

    print(dest_frame_pose_path)

In [ ]:
def write_pose(dest_frame_pose_path, keypoints_augment, keypoints_meta):

    json_list_of_dict = []
    flat_keypoints_augment = [item for sublist in keypoints_augment for item in sublist]
    print(dest_frame_pose_path)
    for frame_no, (pose_aug, pose_meta) in enumerate(
        zip(flat_keypoints_augment, keypoints_meta)
    ):
        json_dict = {}
        json_dict["frameNum"] = frame_no
        for idx, (coord, meta) in enumerate(zip(pose_aug, pose_meta)):
            player = meta[1]
            xy = [coord.x, coord.y, 1.0]
            json_dict.setdefault(player, []).extend(xy)

        json_list_of_dict.append(json_dict)

    name = os.path.split(dest_frame_pose_path)[-1]
    save_path = os.path.join(dest_frame_pose_path, name + ".json")
    with open(save_path, "w") as file:
        json.dump(eval(str(json_list_of_dict)), file, ensure_ascii=False, indent=4)


#     print(json_list_of_dict)

In [ ]:
dest_dir = "./aug_hockey_dataset2"
SERIES = ["Tripping", "Slashing", "No_penalty"]

if not os.path.exists(dest_dir):
    os.makedirs(dest_dir)

In [ ]:
# for series_no in SERIES:
#     count = 0
#     dest_sub_folder = os.path.join(dest_dir, series_no)
#     if not os.path.exists(dest_sub_folder):
#         os.makedirs(dest_sub_folder)

#     for video in os.listdir(osp.join(dataset_dir, series_no)):
#         with open(osp.join(dataset_dir, series_no, video, video + ".json"), 'r') as f:
#             pose =  json.load(f)

#         keypoints_augment, frames_augment, keypoints_meta  = Augment(osp.join(dataset_dir, series_no, video), pose)
#         count = count +1

#         dest_frame_pose_path = os.path.join(dest_sub_folder, video + "100")

#         if not os.path.exists(dest_frame_pose_path):
#             os.makedirs(dest_frame_pose_path)

#         write_image(dest_frame_pose_path, frames_augment)
#         write_pose(dest_frame_pose_path, keypoints_augment, keypoints_meta)

#     print(count)

In [ ]:
####number 2

for series_no in SERIES:
    count = 0
    dest_sub_folder = os.path.join(dest_dir, series_no)

    if not os.path.exists(dest_sub_folder):
        os.makedirs(dest_sub_folder)

    for video in os.listdir(osp.join(dataset_dir, series_no)):

        chance = random.uniform(0, 1)
        print(chance)

        print(os.path.join(dataset_dir, series_no, video))

        with open(osp.join(dataset_dir, series_no, video, video + ".json"), "r") as f:
            pose = json.load(f)

        #             a  = Augment_2(osp.join(dataset_dir, series_no, video), pose)
        keypoints_augment, frames_augment, keypoints_meta = Augment_2(
            osp.join(dataset_dir, series_no, video), pose
        )
        count = count + 1

        dest_frame_pose_path = os.path.join(dest_sub_folder, video + "200")

        if not os.path.exists(dest_frame_pose_path):
            os.makedirs(dest_frame_pose_path)

        write_image(dest_frame_pose_path, frames_augment)
        write_pose(dest_frame_pose_path, keypoints_augment, keypoints_meta)

    print(count)